# Test 03 — Line / IR sensors

Prints sensor states continuously.
Move robot so sensors pass over black tape / line.
Ctrl+C to stop.


In [ ]:
# --- Bootstrap: add repo root + common/lib to sys.path ---
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(repo_root) not in sys.path:
                sys.path.insert(0, str(repo_root))
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return repo_root
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import time
from fast_sdk.infra_red import InfraredSensors

ir = InfraredSensors()
ir.address = 0x77  # from your i2c scan
print('IR addr:', hex(ir.address))

print('Move robot over tape. Ctrl+C to stop.')
try:
    while True:
        vals = ir.read_sensor_data()
        bar = ''.join('■' if v else '·' for v in vals)
        print(bar, vals)
        time.sleep(0.1)
except KeyboardInterrupt:
    print('Stopped')